<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# **Procesamiento de Lenguaje Natural**
# **Desafío 1: Vectorización de texto**

> **Carrera de Especialización en Inteligencia Artificial, Facultad de Ingeniería**
>
> **Universidad de Buenos Aires, Junio de 2024**
>
> Edgar David Guarin Castro (davidg@marketpsychdata.com)

En el presente trabajo se realizan algunos análisis en torno a la vectorización de texto y los modelos de clasificación Naïve Bayes usando el dataset 20 newsgroups.

## **0. Importando librerías**

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random
from IPython.display import display, HTML

## **1. Cargando los datos**

In [2]:
#-----------------
# cargamos los datos (ya separados de forma predeterminada en train y test)
#-----------------
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## **2. Vectorización**

In [3]:
#-----------------
# instanciamos un vectorizador
#-----------------
tfidfvect = TfidfVectorizer()

In [4]:
#-----------------
# en el atributo `data` accedemos al texto
#-----------------
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
#-----------------
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
#-----------------
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
#-----------------
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
#-----------------
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
#-----------------
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
#-----------------
tfidfvect.vocabulary_['car']

25775

In [8]:
#-----------------
# es muy útil tener el diccionario opuesto que va de índices a términos
#-----------------
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
#-----------------
# en `y_train` guardamos los targets que son enteros
#-----------------
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
#-----------------
# hay 20 clases correspondientes a los 20 grupos de noticias
#-----------------
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## **3. Similaridad de documentos**

In [11]:
#-----------------
# Veamos similaridad de documentos. Tomemos algún documento
#-----------------
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
#-----------------
# midamos la similaridad coseno con todos los documentos de train
#-----------------
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
#-----------------
# podemos ver los valores de similaridad ordenados de mayor a menos
#-----------------
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
#-----------------
# y a qué documentos corresponden
#-----------------
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [15]:
#-----------------
# los 5 documentos más similares:
#-----------------
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
#-----------------
# el documento original pertenece a la clase:
#-----------------
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
#-----------------
# y los 5 más similares son de las clases:
#-----------------
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


## **4. Modelo de clasificación Naïve Bayes**

In [18]:
#-----------------
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
#-----------------
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
#-----------------
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
#-----------------
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
#-----------------
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
#-----------------
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

## **5. Desafío**

**Tarea 1:** Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**Tarea 2:** Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**Tarea 3:** Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


## **Solución**

### **Tarea 1:**
Para seleccionar los 5 documentos al azar, se usa la librería random para seleccionar los índices de dichos documentos. Se usa también una semilla en la función random para seleccionar siempre los mismos 5 documentos aleatorios. Estos documentos son impresos posteriormente junto con sus documentos más similares como se muestra a continuación:


In [44]:
#-----------------
# Estableciendo una semilla aleatoria fija
#-----------------
random.seed(60)

#-----------------
# Seleccionando 5 documentos al azar
#-----------------
random_indices = random.sample(range(len(newsgroups_train.data)), 5)

for idx in random_indices:
    html_output = f"<h1>Documento Seleccionado: {idx}</h1>"
    html_output += f"<p><strong>Texto:</strong> {newsgroups_train.data[idx]}</p>"
    html_output += f"<p><strong>Etiqueta:</strong> {newsgroups_train.target_names[y_train[idx]]}</p>"

    #-----------------
    # Calculanado la similitud coseno con todos los documentos
    #-----------------
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    
    #-----------------
    # Ordenando los documentos por similitud (excluyendo el documento en sí mismo)
    #-----------------
    most_similar_indices = np.argsort(cossim)[::-1][1:6]
    
    html_output += f"<h2>Documentos más similares al documento {idx}:</h2>"
    html_output += "<ul>"
    for sim_idx in most_similar_indices:
        similarity = cossim[sim_idx]
        html_output += f"<li><strong>Índice:</strong> {sim_idx} (Similitud: {similarity:.4f})"
        html_output += f"<p><strong>Texto:</strong> {newsgroups_train.data[sim_idx]}</p>"
        html_output += f"<p><strong>Etiqueta:</strong> {newsgroups_train.target_names[y_train[sim_idx]]}</p>"
    html_output += "</ul>"
    html_output += "<hr>"
    display(HTML(html_output))
        

En los ejemplos anteriores, se ve que las similitudes son relativamente bajas en la mayoría de los casos. Esto hace que las etiquetas de algunos documentos no sean exactamente las mismas a las del documento seleccionado. Incluso aparecen documentos sobre temas no relacionados con el documento seleccionado.

Sin embargo, y como es de esperarse, cuando las similitudes son altas como ocurre en el caso del documento 4643, las etiquetas de los documentos coinciden, lo cual puede ser favorecido en parte a la extensión similar de los documentos.

## **Tarea 2**
Se procede en la celda siguiente con el entrenamiento del modelo de clasificación Naïve Bayes:

In [45]:
#-----------------
# Entrenamiento del clasificador
#-----------------
clf = MultinomialNB()
clf.fit(X_train, y_train)

#-----------------
# Transformación del conjunto de prueba
#-----------------
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

#-----------------
# Predicción y evaluación
#-----------------
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1 Score: {f1}")

F1 Score: 0.5854345727938506


El entrenamiento inicial muestra que el F1 score es de 0.585.

Para mejorar este valor, se propone a continuación hacer una búsqueda de hiperparámetros, experimentando con diferentes parámetros de instanciación del TfidfVectorizer tales como:
- max_df: Ignora términos que tienen una frecuencia de documentos superior a este umbral.
- min_df: Ignora términos que tienen una frecuencia de documentos inferior a este umbral.
- ngram_range: Considera n-gramas en lugar de solo unigramas.
- stop_words: Elimina palabras comunes que no aportan mucho valor (como artículos y preposiciones).

También se experimenta ajustando el valor alpha del modelo, el cual representa un parámetro de suavizado.

Para realizar el experimento, se configura un pipeline con TfidfVectorizer y MultinomialNB, y se usa GridSearchCV para encontrar los mejores parámetros. Una vez que se encuentran los mejores parámetros, el modelo se evalúa en el conjunto de prueba:

In [47]:
#-----------------
# Definiendo el pipeline
#-----------------
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

#-----------------
# Definiendo los parámetros para la búsqueda de hiperparámetros
#-----------------
param_grid = {
    'tfidf__max_df': [0.75, 0.85, 0.95],
    'tfidf__min_df': [1, 2, 5],
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'tfidf__stop_words': [None, 'english'],
    'clf__alpha': [0.1, 0.5, 1.0]
}

#-----------------
# Buscando los hiperparámetros con validación cruzada
#-----------------
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(newsgroups_train.data, newsgroups_train.target)

#-----------------
# Imprimiendo los mejores parámetros y el mejor F1-score
#-----------------
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor F1-score de validación: {grid_search.best_score_}")

#-----------------
# Evaluando el mejor modelo en el conjunto de prueba
#-----------------
best_model = grid_search.best_estimator_
y_pred = best_model.predict(newsgroups_test.data)
f1 = f1_score(newsgroups_test.target, y_pred, average='macro')
print(f"F1 Score en el conjunto de prueba: {f1}")

Mejores parámetros: {'clf__alpha': 0.1, 'tfidf__max_df': 0.75, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': 'english'}
Mejor F1-score de validación: 0.7467715176219777
F1 Score en el conjunto de prueba: 0.6729541516556302


Los mejores parámetros obtenidos usando este método permiten observar que el F1 Score del modelo mejoró un 15%, en comparación con la celda anterior donde no se cambiaron los parámetros de TfidfVectorizer y del modelo.

Usando dichos parámetros, se prueban a continuación los modelos MultinomialNB y ComplementNB:

In [48]:
#-----------------
# Parámetros óptimos obtenidos
#-----------------
best_params = {
    'tfidf__max_df': 0.75,
    'tfidf__min_df': 1,
    'tfidf__ngram_range': (1, 2),
    'tfidf__stop_words': 'english',
    'clf__alpha': 0.1
}

#-----------------
# Evaluando MultinomialNB
#-----------------
pipeline_mnb = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=best_params['tfidf__max_df'], 
                              min_df=best_params['tfidf__min_df'],
                              ngram_range=best_params['tfidf__ngram_range'],
                              stop_words=best_params['tfidf__stop_words'])),
    ('clf', MultinomialNB(alpha=best_params['clf__alpha']))
])

pipeline_mnb.fit(newsgroups_train.data, newsgroups_train.target)
y_pred_mnb = pipeline_mnb.predict(newsgroups_test.data)
f1_mnb = f1_score(newsgroups_test.target, y_pred_mnb, average='macro')
print(f"F1 Score con MultinomialNB: {f1_mnb}")

#-----------------
# Evaluando ComplementNB
#-----------------
pipeline_cnb = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=best_params['tfidf__max_df'], 
                              min_df=best_params['tfidf__min_df'],
                              ngram_range=best_params['tfidf__ngram_range'],
                              stop_words=best_params['tfidf__stop_words'])),
    ('clf', ComplementNB(alpha=best_params['clf__alpha']))
])

pipeline_cnb.fit(newsgroups_train.data, newsgroups_train.target)
y_pred_cnb = pipeline_cnb.predict(newsgroups_test.data)
f1_cnb = f1_score(newsgroups_test.target, y_pred_cnb, average='macro')
print(f"F1 Score con ComplementNB: {f1_cnb}")

F1 Score con MultinomialNB: 0.6729541516556302
F1 Score con ComplementNB: 0.7097526747834789


Así, se tiene que el mejor modelo es ComplementNB, ya que posee el F1 score más alto. Esto se debe a que, a diferencia de MultinomialNB, ComplementNB ajusta las probabilidades de las clases para corregir el sesgo introducido por la diferencia en la frecuencia de características entre las clases. Esto permite que el modelo sea más robusto a la hora de clasificar datos desequilibrados y minimizar el impacto de clases mayoritarias.

## **Tarea 3**
Para transponer la matriz documento-término obtenida con TfidfVectorizer y convertirla en una matriz término-documento, se utiliza la transposición de matrices de NumPy como se muestra en la siguiente celda:

In [50]:
#-----------------
# Vectorización
#-----------------
tfidfvect = TfidfVectorizer(max_df=0.75, min_df=1, ngram_range=(1, 2), stop_words='english')
X_train = tfidfvect.fit_transform(newsgroups_train.data)

#-----------------
# Transponer la matriz documento-término a término-documento
#-----------------
X_train_transposed = X_train.transpose()

#-----------------
# Mostrar la forma de la matriz original y la transpuesta
#-----------------
print("Forma de la matriz documento-término:", X_train.shape)
print("Forma de la matriz término-documento:", X_train_transposed.shape)

Forma de la matriz documento-término: (11314, 943737)
Forma de la matriz término-documento: (943737, 11314)


Luego de transponer la matriz, se hace un análisis de la similitud entre 5 palabras específicas y las demás palabras de la matriz.

Para ello, se crea un diccionario de términos para mapear términos a sus índices correspondientes y así encontrar los índices de las palabras seleccionadas. Con esta información, se calcula la similitud coseno y se muestran los resultados:

In [51]:
#-----------------
# Creando un diccionario de términos
#-----------------
terms = tfidfvect.get_feature_names_out()
term_indices = {term: idx for idx, term in enumerate(terms)}

#-----------------
# Seleccionando 5 palabras específicas
#-----------------
selected_words = ['car', 'computer', 'health', 'space', 'education']
selected_indices = [term_indices[word] for word in selected_words]

for idx in selected_indices:
    word = terms[idx]
    html_output = f"<h1>Palabra: {word}</h1>"
    
    #-----------------
    # Calculando similitud coseno con todas las palabras
    #-----------------
    cossim = cosine_similarity(X_train_transposed[idx], X_train_transposed)[0]
    
    #-----------------
    # Ordenando las palabras por similitud (excluyendo la palabra en sí misma)
    #-----------------
    most_similar_indices = np.argsort(cossim)[::-1][1:6]
    
    html_output += "<h3>Palabras más similares:</h3>"
    html_output += "<ul>"
    for sim_idx in most_similar_indices:
        similar_word = terms[sim_idx]
        similarity = cossim[sim_idx]
        html_output += f"<li><strong>Palabra:</strong> {similar_word} (Similitud: {similarity:.4f})</li>"
    html_output += "</ul>"
    html_output += "<hr>"
    display(HTML(html_output))

Los resultados indican que las palabras seleccionadas hacen parte del corpus de documentos, ya que para cada una de ellas fueron encontradas otras palabras similares.

Debido a que el TfidfVectorizer se configuró para considerar n-gramas de longitud 1 a 2 (ngram_range=(1, 2)), el cálculo de similitud considera tanto unigramas (palabras individuales) como bigramas (pares de palabras). Por este motivo, se observan pares de palabras en algunos casos como palabras similares a la escogida.

Algunos de estos bigramas muestran también cuáles son las combinaciones más comunes que contienen la palabra escogida como car car, new car, computer science, computer graphics, etc.

Por otro lado, las similitudes obtenidas siempre se encuentran por debajo de 0.7, lo que indica que el corpus es realmente diverso en relación a los temas que contiene como es el caso de 20 Newsgroups. Esto hace que las palabras aparezcan en diferentes contextos, reduciendo así su similitud.

**NOTA:**
Scripts generados con ayuda de ChatGPT 4o
